# ESM-2

## 1.引用模块

In [1]:
import numpy as np
import pandas as pd
import torch
import os
from tqdm.auto import tqdm
from datasets import Dataset
from torch.utils.data import DataLoader
os.chdir("D:/WorkPath/PycharmProjects/MutTm-pred")
from Dataset.Process4Dataset.DatasetCeator4PonDT import Dataset4MutTm

## 2.导入数据

In [2]:
dataset = Dataset4MutTm(package_path=r"D:\WorkPath\PycharmProjects\MutTm-pred\Dataset\Process4Dataset",
                        train_dataset_path=r"D:\WorkPath\PycharmProjects\MutTm-pred\Dataset\BasicData\ProThermDB\pH-Tm\excllent_ProThermDB_Training.csv",
                        test_dataset_path=r"D:\WorkPath\PycharmProjects\MutTm-pred\Dataset\BasicData\ProThermDB\pH-Tm\excllent_ProThermDB_Testing.csv",
                        training_version="ProThermDB_withpHTm",
                        testing_version="ProThermDBTest_withpHTm",
                        selected_columns=["UniProt_ID", "Mutation", "pH", "Tm", "ΔTm"],
                        mult_mode="Average",
                        features=["neighbor", "sift4g", "rpm", "param", "aaindex", "group", "GO"],
                        R_path=r"C:\Program Files\R\R-4.3.2",
                        context_length=50)

===正在从训练集版本为ProThermDB_withpHTm、训练集版本为ProThermDBTest_withpHTm的原始数据集中进行数据清洗和生物特征提取工作===
1.预处理训练集数据...
删除数据缺失行及非法行共计1行
丢弃pH/Tm的缺失值
前一数据集采用了后一数据集中的0条数据，现已删除
获取[序列]信息.....该数据集已经经过处理，直接使用缓存文件
删除了0个非法长度的蛋白质，当前蛋白质长度被限制在(50, 5000)
2.预处理测试集数据...
删除数据缺失行及非法行共计0行
丢弃pH/Tm的缺失值
获取[序列]信息.....该数据集已经经过处理，直接使用缓存文件
删除了0个非法长度的蛋白质，当前蛋白质长度被限制在(50, 5000)
3.为训练集数据提取生物特征...
获取[neighbor特征].....该数据集已经经过处理，直接使用缓存文件
获取[aaindex特征].....获取[group特征].....该数据集已经经过处理，直接使用缓存文件
获取[pssm]特征.....该数据集已经经过处理，直接使用缓存文件
获取[sift4g]特征.....该数据集已经经过处理，直接使用缓存文件
获取[param]特征.....该数据集已经经过处理，直接使用缓存文件
4.为测试集数据提取生物特征...
获取[neighbor特征].....该数据集已经经过处理，直接使用缓存文件
获取[aaindex特征].....获取[group特征].....该数据集已经经过处理，直接使用缓存文件
获取[pssm]特征.....该数据集已经经过处理，直接使用缓存文件
获取[sift4g]特征.....该数据集已经经过处理，直接使用缓存文件
获取[param]特征.....该数据集已经经过处理，直接使用缓存文件
5.利用GO富集分析从训练集和测试集中提取LR与PA值
获取[GO]特征.....

100%|██████████| 218/218 [00:00<00:00, 10381.00it/s]


6.从全数据集中提取生物特征集、标签集和基本信息集...
7.数据清洗和生物特征提取工作完成==>


## 3.导入预训练模型

In [3]:
from transformers import AutoTokenizer, AutoModel
modelPath = "DeepLearning/EmbeddingModels/ESM-2/esm2_t30_150M_UR50D"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=modelPath)
model = AutoModel.from_pretrained(modelPath).cuda()

Some weights of EsmModel were not initialized from the model checkpoint at DeepLearning/EmbeddingModels/ESM-2/esm2_t30_150M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 构建数据集

In [4]:
train_wild_tokenized = tokenizer(dataset.train_basic_set["WildSeq"].to_list(), return_tensors="pt", padding=False)
test_wild_tokenized = tokenizer(dataset.test_basic_set["WildSeq"].to_list(), return_tensors="pt", padding=False)
train_mutant_tokenized = tokenizer(dataset.train_basic_set["MutantSeq"].to_list(), return_tensors="pt", padding=False)
test_mutant_tokenized = tokenizer(dataset.test_basic_set["MutantSeq"].to_list(), return_tensors="pt", padding=False)
train_labels = np.array(dataset.train_label_set)
test_labels = np.array(dataset.test_label_set)

train_dataset = Dataset.from_dict({"wild_input_ids": train_wild_tokenized["input_ids"],
                                   "wild_attention_mask": train_wild_tokenized["attention_mask"],
                                   "labels": train_labels,
                                   "mutant_input_ids": train_mutant_tokenized["input_ids"],
                                   "mutant_attention_mask": train_mutant_tokenized["attention_mask"]})
test_dataset = Dataset.from_dict({"wild_input_ids": test_wild_tokenized["input_ids"],
                                  "wild_attention_mask": test_wild_tokenized["attention_mask"],
                                  "labels": test_labels,
                                  "mutant_input_ids": test_mutant_tokenized["input_ids"],
                                  "mutant_attention_mask": test_mutant_tokenized["attention_mask"]})

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, drop_last=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=True, drop_last=True)

## 5.自定义模型

In [5]:
from torch.nn import init
from torch import nn
from torch.nn import functional as F
import math

### 5.1.前馈多头注意力网络

In [6]:
# PreAttention类
class Attention(nn.Module):
    # InitializationDataset:https://github.com/WenYanger/Contextual-Attention/blob/master/Attention_Pytorch.py
    def __init__(self, input_shape):
        super(Attention, self).__init__()

        self.max_len = input_shape[1]
        self.emb_size = input_shape[2]

        # Change double to float
        self.weight = nn.Parameter(torch.Tensor(self.emb_size, 1))
        self.bias = nn.Parameter(torch.Tensor(self.max_len, 1))

        self.reset_parameters()

    def reset_parameters(self):
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)

    def extra_repr(self):
        return 'max_len={}, emb_size={}'.format(
            self.max_len, self.emb_size
        )

    def forward(self, inp, mask=None):
        # Here    sr should be [batch_size, time_step, emb_size]
        #      mask should be [batch_size, time_step, 1]
        W_bs = self.weight.unsqueeze(0).repeat(inp.size()[0], 1, 1).float()  # Copy the Attention Matrix for batch_size times
        scores = torch.bmm(inp, W_bs)  # Dot product between input and attention matrix
        scores = torch.tanh(scores)

        # scores = Cal_Attention()(sr, self.weight, self.bias)

        if mask is not None:
            mask = mask.long()
            scores = scores.masked_fill(mask == 0, -1e9)

        a_ = F.softmax(scores.squeeze(-1), dim=-1)
        a = a_.unsqueeze(-1).repeat(1, 1, inp.size()[2])

        weighted_input = inp * a

        output = torch.sum(weighted_input, dim=1)

        return output  # 不返回权重矩阵

### 5.2.Main Model

In [7]:
# 特征提取网络类
class PonDT(nn.Module):

    def __init__(self,
                 length,
                 embedding_model):
        super(PonDT, self).__init__()

        self.embedding_model = embedding_model

        # 特征提取模块
        self.Input_Feature_Module = nn.Sequential(
            Attention([None, length, 640]),
            nn.BatchNorm1d(640, dtype=torch.float32),
            nn.Dropout(0.3)
        )

        # 回归模块
        self.Fussion_Module = nn.Sequential(
            nn.Linear(1280, 512, dtype=torch.float32), nn.ReLU(inplace=True),
            nn.Linear(512, 256, dtype=torch.float32), nn.ReLU(inplace=True),
            nn.Linear(256, 128, dtype=torch.float32), nn.ReLU(inplace=True),
            nn.Linear(128, 32, dtype=torch.float32), nn.ReLU(inplace=True),
            nn.Linear(32, 1, dtype=torch.float32)
        )

    def forward(self, wild_ids, wild_mask, mutant_ids, mutant_mask):
        # 预训练特征模块
        wild_output = self.embedding_model(input_ids=wild_ids, attention_mask=wild_mask).last_hidden_state
        mutant_output = self.embedding_model(input_ids=mutant_ids, attention_mask=mutant_mask).last_hidden_state
        # 特征提取模块
        wild_output = self.Input_Feature_Module(wild_output)
        mutant_output = self.Input_Feature_Module(mutant_output)

        # 拼接机器学习特征与深度特征
        combination = torch.cat((wild_output, mutant_output), dim=1)

        # MLP回归预测
        out = self.Fussion_Module(combination)

        # output
        return out

## 定义训练参数

In [8]:
device = torch.device("cuda")
pondt = PonDT(length=52, embedding_model=model).to(device)
loss_fn = torch.nn.L1Loss().to(device)
optimizer = torch.optim.Adam(params=pondt.parameters(),
                             lr=1e-3,
                             betas=(0.9, 0.999),
                             eps=1e-3)
num_epochs = 1000
num_training_steps = num_epochs * len(train_loader)

## 训练

In [15]:
with tqdm(range(num_training_steps)) as progress_bar:
    model.train()
    for epoch in range(num_epochs):
        for batch in train_loader:
            
            wild_input_ids = torch.stack(batch["wild_input_ids"], dim=1).to(device)
            wild_attention_mask = torch.stack(batch["wild_attention_mask"], dim=1).to(device)
            mutant_input_ids = torch.stack(batch["mutant_input_ids"], dim=1).to(device)
            mutant_attention_mask = torch.stack(batch["mutant_attention_mask"], dim=1).to(device)
            outputs = pondt(wild_input_ids, wild_attention_mask, mutant_input_ids, mutant_attention_mask)
            labels = batch["labels"].to(device).unsqueeze(1)
            loss = loss_fn(outputs, labels)
            
            optimizer.zero_grad()
            loss.requires_grad_(True)
            loss.backward()
            optimizer.step()
            progress_bar.update(1)

  0%|          | 0/1320 [00:00<?, ?it/s]

In [16]:
a = torch.Tensor([3, 2, 1])


In [17]:
a.size()

torch.Size([3])

In [18]:
b = a.unsqueeze(1)

In [20]:
b.size()

torch.Size([3, 1])

In [21]:
len(a.size()), len(b.size())

(1, 2)

In [24]:
new = batch["wild_input_ids"]

In [25]:
old = torch.stack(batch["wild_input_ids"], dim=1)

In [27]:
type(new)

list

In [28]:
type(old)

torch.Tensor